In [1]:
%load_ext autoreload
%autoreload 2

import molpy as mp
import numpy as np
from pathlib import Path

data_path = Path("data/waterbox")

In [2]:
class SPCE(mp.AtomicStructure):

    def __init__(self, name="spce"):
        super().__init__(name=name)
        o = self.def_atom(
            name="o", molid="$w", type="O", q=-0.8476, xyz=[0.00000, 0.00000, 0.00000]
        )
        h1 = self.def_atom(
            name="h1",
            molid="$w",
            type="H",
            q=0.4238,
            xyz=[0.8164904, 0.5773590, 0.00000],
        )
        h2 = self.def_atom(
            name="h2",
            molid="$w",
            type="H",
            q=0.4238,
            xyz=[-0.8164904, 0.5773590, 0.00000],
        )
        self.def_bond(o, h1)
        self.def_bond(o, h2)
        
        # Generate angles using the new gen_topo_items method
        angles = self.gen_topo_items(is_angle=True)
        print(f"Generated {len(angles)} angles for SPCE water molecule")

In [3]:
ff = mp.ForceField(name="spce", unit="real")
atomstyle = ff.def_atomstyle("full")
o_type = atomstyle.def_type("O", mass=15.999)
h_type = atomstyle.def_type("H", mass=1.008)

bondstyle = ff.def_bondstyle("harmonic")
bondstyle.def_type(
    o_type, h_type, k=1000.0, r0=1.0
)

anglestyle = ff.def_anglestyle("harmonic")
anglestyle.def_type(
    h_type, o_type, h_type, k=1000.0, theta0=109.47
)

pairstyle = ff.def_pairstyle("lj/charmm/coul/long", inner=9.0, outer=10.0, cutoff=10.0, mix="arithmetic")
pairstyle.def_type(
    o_type, o_type, epsilon=0.1554, sigma=3.1656
)
pairstyle.def_type(
    h_type, h_type, epsilon=0.0, sigma=0.0
)

<PairType: H-H>

In [4]:
typifier = mp.typifier.ForceFieldTypifier(forcefield=ff)
spec = typifier.typify(SPCE())

Generated 1 angles for SPCE water molecule


In [5]:

system = mp.System()
system.set_forcefield(ff)

# 设置盒子大小 (3x3x3 的小盒子用于测试)
box_size = 10.0
system.def_box(np.diag([box_size, box_size, box_size]))

# 创建 3x3x3 = 27 个水分子
n_molecules = 0
spacing = 3.0  # 分子间距

for i in range(3):
    for j in range(3):
        for k in range(3):
            # 创建新的 SPCE 分子
            water_mol = spec()  # 使用 typified 的分子
            
            # 用 SpatialWrapper 包装并移动到指定位置
            spatial_water = mp.SpatialWrapper(water_mol)
            position = [i * spacing, j * spacing, k * spacing]
            spatial_water.move(position)
            
            # 添加到系统
            system.add_struct(water_mol)
            n_molecules += 1

print(f"创建了 {n_molecules} 个水分子")

# 检查系统内容 - 使用正确的属性访问方式
if hasattr(system, '_struct'):
    total_atoms = sum(len(struct.atoms) for struct in system._struct)
    total_bonds = sum(len(struct.bonds) for struct in system._struct)
    total_angles = sum(len(struct.angles) for struct in system._struct)
    
    print(f"系统包含 {total_atoms} 个原子")
    print(f"系统包含 {total_bonds} 个键")
    print(f"系统包含 {total_angles} 个角")
else:
    print("系统结构信息不可用")

# write to lammps

创建了 27 个水分子
系统包含 81 个原子
系统包含 54 个键
系统包含 27 个角


In [ ]:
# Direct export to LAMMPS
print("=== Direct LAMMPS Export ===")

# Create output directory
data_path.mkdir(exist_ok=True)

# Convert system to frame
frame = system.to_frame()
print(f"Frame generated with {len(frame['atoms'])} atoms")

# Fix xarray to pandas conversion for LAMMPS writer
if hasattr(frame['atoms'], 'to_dataframe'):
    atoms_df = frame['atoms'].to_dataframe().reset_index()
    frame['atoms'] = atoms_df
    print("Converted atoms data to pandas DataFrame")

# Export LAMMPS data file
data_file = data_path / "water_box.data"
mp.io.write_lammps_data(data_file, frame)
print(f"✅ LAMMPS data file: {data_file}")

# Export LAMMPS forcefield file  
ff_file = data_path / "water_box.ff"
mp.io.write_lammps_forcefield(ff_file, frame)
print(f"✅ LAMMPS forcefield file: {ff_file}")

# Show created files
print(f"\nCreated files:")
for f in data_path.glob("water_box.*"):
    print(f"  - {f.name} ({f.stat().st_size} bytes)")

print(f"\n🎉 LAMMPS files exported successfully!")
print(f"📁 Location: {data_path}")
print(f"💧 System: {n_molecules} water molecules, {total_atoms} atoms")

=== Direct LAMMPS Export ===
Frame generated with 6 atoms
